## Linear Classifier in TensorFlow 
Using Low Level API in Eager Execution mode

### Load tensorflow

In [1]:
import tensorflow as tf
tf.enable_eager_execution()

In [2]:
%tensorflow_version 2.x

TensorFlow is already loaded. Please restart the runtime to change versions.


In [0]:
#Enable Eager Execution if using tensflow version < 2.0
#From tensorflow v2.0 onwards, Eager Execution will be enabled by default


### Collect Data

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
import pandas as pd

In [0]:
data = pd.read_csv('/content/drive/My Drive/Colab Notebooks/Data/prices.csv')



### Check all columns in the dataset

In [6]:
data.columns

Index([u'date', u'symbol', u'open', u'close', u'low', u'high', u'volume'], dtype='object')

In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 851264 entries, 0 to 851263
Data columns (total 7 columns):
date      851264 non-null object
symbol    851264 non-null object
open      851264 non-null float64
close     851264 non-null float64
low       851264 non-null float64
high      851264 non-null float64
volume    851264 non-null float64
dtypes: float64(5), object(2)
memory usage: 45.5+ MB


### Drop columns `date` and  `symbol`

In [0]:
data.drop(['date','symbol'],axis =1,inplace = True)

In [9]:
data.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2163600.0
1,125.239998,119.980003,119.940002,125.540001,2386400.0
2,116.379997,114.949997,114.930000,119.739998,2489500.0
3,115.480003,116.620003,113.500000,117.440002,2006300.0
4,117.010002,114.970001,114.089996,117.330002,1408600.0


### Consider only first 1000 rows in the dataset for building feature set and target set
Target 'Volume' has very high values. Divide 'Volume' by 1000,000

In [0]:
data_df = data.iloc[0:1000]

In [11]:
data_df.shape

(1000, 5)

In [12]:
data_df['volume'] = data_df['volume'] / 1000000

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [13]:
data_df.head()

,open,close,low,high,volume
0,123.430000,125.839996,122.309998,126.250000,2.1636
1,125.239998,119.980003,119.940002,125.540001,2.3864
2,116.379997,114.949997,114.930000,119.739998,2.4895
3,115.480003,116.620003,113.500000,117.440002,2.0063
4,117.010002,114.970001,114.089996,117.330002,1.4086


### Divide the data into train and test sets

In [0]:
y = data_df['volume']
x = data_df.drop(['volume'], axis = 1)

In [15]:
x.shape, y.shape

((1000, 4), (1000,))

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
train_x,test_x,train_y,test_y = train_test_split(x,y,test_size = 0.3, random_state =5)

#### Convert Training and Test Data to numpy float32 arrays


In [0]:
import numpy as np
train_x =np.array(train_x).astype('float32')
test_x = np.array(test_x).astype('float32')
train_y =np.array(train_y).astype('float32')
test_y = np.array(test_y).astype('float32')

In [0]:
train_y=train_y.reshape(-1,1)

In [20]:
train_y.shape

(700, 1)

### Normalize the data
You can use Normalizer from sklearn.preprocessing

In [0]:
from sklearn.preprocessing import Normalizer
transformer = Normalizer()
train_x = transformer.fit_transform(train_x)
test_x = transformer.transform(test_x)

## Building the Model in tensorflow

1.Define Weights and Bias, use tf.zeros to initialize weights and Bias

In [0]:
#We are initializing weights and Bias of (Input-Hidden Layer)
w1 = tf.random_normal(shape=(4,2))
b1 = tf.zeros(shape=(2)) 


In [0]:
#We are initializing weights and Bias of (Hidden-Output)
w2 = tf.random_normal(shape=(2,1))
b2 = tf.zeros(shape=(1))

2.Define a function to calculate prediction

In [0]:
def prediction(x, w1, b1,w2,b2):
    
    xw_matmul = tf.matmul(x, w1)
    net1 = tf.add(xw_matmul, b1)
    y=tf.sigmoid(net1)
    net2=tf.matmul(y, w2)+b2
    out=tf.sigmoid(net2)
    return net2,out

3.Loss (Cost) Function [Mean square error]

In [0]:
def loss(predicted_y, desired_y):
  return tf.reduce_mean(tf.square(predicted_y - desired_y))

4.Function to train the Model

1.   Record all the mathematical steps to calculate Loss
2.   Calculate Gradients of Loss w.r.t weights and bias
3.   Update Weights and Bias based on gradients and learning rate to minimize loss

In [0]:
def train(train_x, train_y, w1, b1,w2,b2, learning_rate=0.01):
    
    #Record mathematical operations on 'tape' to calculate loss
    with tf.GradientTape() as t:
        
        t.watch([w1,b1,w2,b2])
        
        net2,current_prediction = prediction(train_x, w1, b1,w2,b2)
        #current_loss =tf.nn.sigmoid_cross_entropy_with_logits(labels = train_y,logits =net2 )
        current_loss = loss(net2, train_y)
        print(current_loss)
    
    #Calculate Gradients for Loss with respect to Weights and Bias
    dw1,db1,dw2,db2 = t.gradient(current_loss,[w1, b1,w2,b2])
    
    #Update Weights at output layer
    w2 = w2 - learning_rate*dw2
    b2 = b2 - learning_rate*db2
    
     #Update Weights at hidden layer
    w1 = w1 - learning_rate*dw1
    b1 = b1 - learning_rate*db1

    #print("Loss at step {:d}: {:.3f}".format(i, currentloss)
    
    return w1, b1,w2,b2


## Train the model for 100 epochs 
1. Observe the training loss at every iteration
2. Observe Train loss at every 5th iteration

In [27]:
for i in range(100):
    
    w1,b1,w2,b2 = train(train_x, train_y, w1, b1,w2,b2)

tf.Tensor(251.61565, shape=(), dtype=float32)
tf.Tensor(250.29364, shape=(), dtype=float32)
tf.Tensor(249.0402, shape=(), dtype=float32)
tf.Tensor(247.85219, shape=(), dtype=float32)
tf.Tensor(246.72705, shape=(), dtype=float32)
tf.Tensor(245.66234, shape=(), dtype=float32)
tf.Tensor(244.65567, shape=(), dtype=float32)
tf.Tensor(243.70502, shape=(), dtype=float32)
tf.Tensor(242.80826, shape=(), dtype=float32)
tf.Tensor(241.96349, shape=(), dtype=float32)
tf.Tensor(241.16866, shape=(), dtype=float32)
tf.Tensor(240.42194, shape=(), dtype=float32)
tf.Tensor(239.7215, shape=(), dtype=float32)
tf.Tensor(239.06535, shape=(), dtype=float32)
tf.Tensor(238.45168, shape=(), dtype=float32)
tf.Tensor(237.87866, shape=(), dtype=float32)
tf.Tensor(237.34442, shape=(), dtype=float32)
tf.Tensor(236.84698, shape=(), dtype=float32)
tf.Tensor(236.38464, shape=(), dtype=float32)
tf.Tensor(235.95544, shape=(), dtype=float32)
tf.Tensor(235.55762, shape=(), dtype=float32)
tf.Tensor(235.18953, shape=(), dtype

In [28]:
for i in range(0,100,5):
    print(i)
    w1,b1,w2,b2 = train(train_x, train_y, w1, b1,w2,b2)

0
tf.Tensor(231.21446, shape=(), dtype=float32)
5
tf.Tensor(231.2144, shape=(), dtype=float32)
10
tf.Tensor(231.21422, shape=(), dtype=float32)
15
tf.Tensor(231.21408, shape=(), dtype=float32)
20
tf.Tensor(231.21402, shape=(), dtype=float32)
25
tf.Tensor(231.21397, shape=(), dtype=float32)
30
tf.Tensor(231.21384, shape=(), dtype=float32)
35
tf.Tensor(231.21375, shape=(), dtype=float32)
40
tf.Tensor(231.21375, shape=(), dtype=float32)
45
tf.Tensor(231.21365, shape=(), dtype=float32)
50
tf.Tensor(231.21365, shape=(), dtype=float32)
55
tf.Tensor(231.21364, shape=(), dtype=float32)
60
tf.Tensor(231.21358, shape=(), dtype=float32)
65
tf.Tensor(231.2135, shape=(), dtype=float32)
70
tf.Tensor(231.21344, shape=(), dtype=float32)
75
tf.Tensor(231.21344, shape=(), dtype=float32)
80
tf.Tensor(231.21344, shape=(), dtype=float32)
85
tf.Tensor(231.21341, shape=(), dtype=float32)
90
tf.Tensor(231.2134, shape=(), dtype=float32)
95
tf.Tensor(231.21333, shape=(), dtype=float32)


### Get the shapes and values of W and b

In [29]:
w1

<tf.Tensor: id=6495, shape=(4, 2), dtype=float32, numpy=
array([[ 1.816876  , -0.30552328],
       [ 0.3293008 ,  1.5312587 ],
       [-0.46109727,  0.15483312],
       [ 0.68466216, -1.4926614 ]], dtype=float32)>

In [30]:
w1.shape,w2.shape

(TensorShape([Dimension(4), Dimension(2)]),
 TensorShape([Dimension(2), Dimension(1)]))

In [31]:
w2

<tf.Tensor: id=6489, shape=(2, 1), dtype=float32, numpy=
array([[2.1529672],
       [1.7615798]], dtype=float32)>

In [32]:
b1

<tf.Tensor: id=6498, shape=(2,), dtype=float32, numpy=array([0.49514392, 0.626014  ], dtype=float32)>

In [33]:
b2

<tf.Tensor: id=6492, shape=(1,), dtype=float32, numpy=array([2.17399], dtype=float32)>

### Model Prediction on 1st Examples in Test Dataset

In [0]:
y_pred,current_prediction = prediction(test_x, w1, b1,w2,b2)

In [35]:
y_pred

<tf.Tensor: id=6504, shape=(300, 1), dtype=float32, numpy=
array([[5.112621 ],
       [5.11433  ],
       [5.111449 ],
       [5.11388  ],
       [5.109508 ],
       [5.1121182],
       [5.1124725],
       [5.1098332],
       [5.1125784],
       [5.1130657],
       [5.1131167],
       [5.112302 ],
       [5.1099796],
       [5.11208  ],
       [5.1149874],
       [5.109944 ],
       [5.1138744],
       [5.1107655],
       [5.1136503],
       [5.1117926],
       [5.111127 ],
       [5.1116886],
       [5.1136956],
       [5.114218 ],
       [5.107243 ],
       [5.1124907],
       [5.1137915],
       [5.113505 ],
       [5.1131444],
       [5.114395 ],
       [5.1131983],
       [5.112752 ],
       [5.111543 ],
       [5.111226 ],
       [5.1135616],
       [5.1141634],
       [5.113692 ],
       [5.1142063],
       [5.113824 ],
       [5.1139903],
       [5.113063 ],
       [5.1144686],
       [5.114694 ],
       [5.113035 ],
       [5.114704 ],
       [5.112378 ],
       [5.107429 ],
 

In [36]:
from sklearn.metrics import r2_score
print(r2_score(test_y,y_pred))

-0.002667624433597693


## Classification using tf.Keras

In this exercise, we will build a Deep Neural Network using tf.Keras. We will use Iris Dataset for this exercise.

### Load the given Iris data using pandas (Iris.csv)

In [0]:
iris = pd.read_csv("/content/drive/My Drive/Colab Notebooks/Data/11_Iris.csv")

### Target set has different categories. So, Label encode them. And convert into one-hot vectors using get_dummies in pandas.

In [64]:
iris.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,1,5.1,3.5,1.4,0.2,Iris-setosa
1,2,4.9,3.0,1.4,0.2,Iris-setosa
2,3,4.7,3.2,1.3,0.2,Iris-setosa
3,4,4.6,3.1,1.5,0.2,Iris-setosa
4,5,5.0,3.6,1.4,0.2,Iris-setosa


In [0]:
dummy = pd.get_dummies(iris['Species'])

In [0]:
iris = pd.concat([iris, dummy], axis=1)

In [0]:
iris.drop(labels=['Species'], axis=1, inplace=True)

In [70]:
iris.head()

,Id,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Iris-setosa,Iris-versicolor,Iris-virginica
0,1,5.1,3.5,1.4,0.2,1,0,0
1,2,4.9,3.0,1.4,0.2,1,0,0
2,3,4.7,3.2,1.3,0.2,1,0,0
3,4,4.6,3.1,1.5,0.2,1,0,0
4,5,5.0,3.6,1.4,0.2,1,0,0


In [0]:
iris = iris.values

In [72]:
type(iris)

numpy.ndarray

### Splitting the data into feature set and target set

In [0]:
# split into input (X) and output (Y) variables
X = iris[:,0:4]
y = iris[:,5:8]

###  Building Model in tf.keras

Build a Linear Classifier model  <br>
1.  Use Dense Layer  with input shape of 4 (according to the feature set) and number of outputs set to 3<br> 
2. Apply Softmax on Dense Layer outputs <br>
3. Use SGD as Optimizer
4. Use categorical_crossentropy as loss function 

In [0]:
from keras.models import Sequential
from keras.layers import Dense

In [0]:
tf.disable_eager_execution()

In [88]:
# define base model
model = Sequential()
model.add(Dense(4, input_dim=4, kernel_initializer='normal', activation='softmax'))
model.add(Dense(3, kernel_initializer='normal'))
# Compile model
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])

W1201 12:08:08.498079 140709949142912 module_wrapper.py:139] From /usr/local/lib/python2.7/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W1201 12:08:08.520174 140709949142912 module_wrapper.py:139] From /usr/local/lib/python2.7/dist-packages/keras/backend/tensorflow_backend.py:3295: The name tf.log is deprecated. Please use tf.math.log instead.



### Model Training 

In [0]:
train_x,test_x,train_y,test_y = train_test_split (X,y,test_size = 0.3, random_state =5)

In [96]:
model.fit(train_x, train_y, validation_data=(test_x, test_y), epochs=50,
          batch_size = train_x.shape[0])

W1201 12:12:45.953229 140709949142912 deprecation.py:323] From /usr/local/lib/python2.7/dist-packages/tensorflow_core/python/ops/math_grad.py:1424: where (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W1201 12:12:46.033421 140709949142912 module_wrapper.py:139] From /usr/local/lib/python2.7/dist-packages/keras/backend/tensorflow_backend.py:2741: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W1201 12:12:46.041554 140709949142912 module_wrapper.py:139] From /usr/local/lib/python2.7/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W1201 12:12:46.046971 140709949142912 module_wrapper.py:139] From /usr/local/lib/python2.7/dist-packages/keras/backend/tensorflow_backend.py:190: The name tf.global_variables is deprecate

Train on 105 samples, validate on 45 samples
Epoch 1/50
105/105 [==============================] - 0s 4ms/step - loss: 5.0305 - acc: 0.2857 - val_loss: 5.7676 - val_acc: 0.3333
Epoch 2/50
105/105 [==============================] - 0s 26us/step - loss: 5.7324 - acc: 0.3333 - val_loss: 5.7630 - val_acc: 0.3333
Epoch 3/50
105/105 [==============================] - 0s 47us/step - loss: 5.7282 - acc: 0.3333 - val_loss: 5.7585 - val_acc: 0.3333
Epoch 4/50
105/105 [==============================] - 0s 31us/step - loss: 5.7240 - acc: 0.3333 - val_loss: 5.7541 - val_acc: 0.3333
Epoch 5/50
105/105 [==============================] - 0s 28us/step - loss: 5.7200 - acc: 0.3333 - val_loss: 5.7498 - val_acc: 0.3333
Epoch 6/50
105/105 [==============================] - 0s 27us/step - loss: 5.7161 - acc: 0.3333 - val_loss: 5.7456 - val_acc: 0.3333
Epoch 7/50
105/105 [==============================] - 0s 30us/step - loss: 5.7123 - acc: 0.3333 - val_loss: 5.7415 - val_acc: 0.3333
Epoch 8/50
105/105 [=====

### Model Prediction

In [97]:
model.predict(test_x)

array([[ 1.2839463 , -0.83517134, -2.0794253 ],
       [ 1.2839463 , -0.83517134, -2.0794253 ],
       [ 1.2839463 , -0.83517134, -2.0794253 ],
       [ 1.2839463 , -0.83517134, -2.0794253 ],
       [ 1.2839463 , -0.83517134, -2.0794253 ],
       [ 1.2839463 , -0.83517134, -2.0794253 ],
       [ 1.2839463 , -0.83517134, -2.0794253 ],
       [ 1.2839463 , -0.83517134, -2.0794253 ],
       [ 1.2839463 , -0.83517134, -2.0794253 ],
       [ 1.2839463 , -0.83517134, -2.0794253 ],
       [ 1.2839463 , -0.83517134, -2.0794253 ],
       [ 1.2839463 , -0.83517134, -2.0794253 ],
       [ 1.2839463 , -0.83517134, -2.0794253 ],
       [ 1.2839463 , -0.83517134, -2.0794253 ],
       [ 1.2839463 , -0.83517134, -2.0794253 ],
       [ 1.2839463 , -0.83517134, -2.0794253 ],
       [ 1.2839463 , -0.83517134, -2.0794253 ],
       [ 1.2839463 , -0.83517134, -2.0794253 ],
       [ 1.2839463 , -0.83517134, -2.0794253 ],
       [ 1.2839463 , -0.83517134, -2.0794253 ],
       [ 1.2839463 , -0.83517134, -2.079

### Save the Model

In [0]:
model.save('iris_model-santosh.html')

### Build and Train a Deep Neural network with 2 hidden layer  - Optional - For Practice

Does it perform better than Linear Classifier? What could be the reason for difference in performance?